# Tutorial 2 - Extracción de noticias en medios de prensa internacionales

Ejemplo aplicado al medio: https://www.washingtonpost.com

sintáxis del lenguaje XPath: https://www.w3schools.com/xml/xpath_syntax.asp

## Parte 1: Crawling

Recopilación de las URLs que contienen noticias de prensa

### 1. URL _seed_

In [9]:
SEED_CRAWL = 'https://www.washingtonpost.com/pb/api/v2/render/feature/section/story-list?content_origin=prism-query&url=prism://prism.query/site-articles-only,/politics&offset=5&limit=15'

In [10]:
HEADER = {"authority": "www.washingtonpost.com", "accept": "*/*", "dnt": "1", "x-requested-with": "XMLHttpRequest", "user-agent": "Mozilla/5.0", "sec-fetch-site": "same-origin", "sec-fetch-mode": "cors", "referer": "https://www.washingtonpost.com/politics/?itid=nb_hp_politics"} 

Para iniciar el proceso de crawling, necesitamos una página inicial que llamamos 'SEED'. Es la página que contiene una lista de enlaces hacia noticias.

### 2. Class _Crawler_

La clase _Crawler_ se encarga de enviar una consulta HTTP (GET) a la pagina SEED.  Luego analiza la respuesta HTML para obtener los enlaces HTML.

Se requiere definir la ruta HTML para obtener los enlaces que corresponden a las noticias de prensa.

In [11]:
QUERY_URL = "//div[contains(@class,'story-list-story row item')]//div[@class='story-headline']//a/@href"

Para consultar y analizar la pagina SEED, se utiliza una serie de librerias y se selecciona al azar un navegador web.

In [12]:
import random
import requests
from requests_html import HTMLSession
from requests_html import HTML
import os
import json
import validators

In [13]:
class Crawler:
    
    USER_AGENT_LIST = [
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
        "Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/19.77.34.5 Safari/537.1",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.9 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.36 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"
]

    query_extract_all_href=QUERY_URL

    all_json_to_send = []
    
    def _init_(self):
        pass
    
    def parse(self, response):
        
        self.all_json_to_send = []
        respose_as_json= json.loads(response.html.html)
        html_as_string = respose_as_json['rendering']
        extracted_html = HTML(html=html_as_string)
        all_extracted_href = extracted_html.xpath(self.query_extract_all_href)
        for href in all_extracted_href:
            json_to_send= dict()
            json_to_send["url"]=href
            self.all_json_to_send.append(json_to_send)
        print(self.all_json_to_send)   

    def start_request(self,url,header=dict()):
        session = HTMLSession()
        header['user-agent'] = random.choice(self.USER_AGENT_LIST)
        response = session.get(url,headers=header)        
        return response

### 3. Crawling y parsing

In [16]:
crawler = Crawler()

response = crawler.start_request(SEED_CRAWL, HEADER)

In [17]:
response.html.html

'{"rendering":"<div class=\\"moat-trackable pb-f-theme-normal pb-f-dehydrate-false pb-f-async-false pb-2 pb-feature pb-layout-item pb-f-section-story-list\\" moat-id=\\"section/story-list\\" data-chain-name=\\"no-name\\" data-feature-name=\\"no-name\\" data-feature-id=\\"section/story-list\\" data-pb-fingerprint=\\"no-fingerprint\\" id=\\"\\"><div class=\\"row \\"> <div class=\\"col-md-12 col-lg-11 col-lg-offset-1\\"> <div class=\\"story-list item\\"> <div class=\\"story-list-story row first item\\" data-pb-local-content-id=\\"2f67c892b49f4d1eaa07f48d6f3d62ab93e9fdce\\" itemprop=\\"itemListElement\\" itemscope itemtype=\\"http://schema.org/ListItem\\"> <div class=\\"story-body col-xs-8 col-md-8\\"> <meta itemprop=\\"position\\" content=\\"1\\"/> <div class=\\"story-headline\\"> <div class=\\"moat-trackable pb-f-theme-normal pb-f-dehydrate-false pb-f-async-false pb-feature pb-layout-item pb-f-sharebars-top-share-bar\\" moat-id=\\"sharebars/top-share-bar\\" data-chain-name=\\"no-name\\" 

In [18]:
crawler.parse(response)

[{'url': 'https://www.washingtonpost.com/politics/white-house-signals-stronger-coronavirus-precautions-but-trump-continues-to-resist/2020/10/06/05cd0be4-07e9-11eb-859b-f9c27abe638d_story.html'}, {'url': 'https://www.washingtonpost.com/politics/senate-north-carolina-cunningham-democrats/2020/10/06/103de8fe-07ea-11eb-a166-dc429b380d10_story.html'}, {'url': 'https://www.washingtonpost.com/politics/2020/10/02/visual-timeline-trumps-movements-before-his-positive-coronavirus-test/'}, {'url': 'https://www.washingtonpost.com/politics/2020/10/02/latest-coronavirus-test-results-trumps-advisers-allies/'}, {'url': 'https://www.washingtonpost.com/politics/courts_law/supreme-court-fbi-no-fly-list/2020/10/06/1d814b7c-07ff-11eb-859b-f9c27abe638d_story.html'}, {'url': 'https://www.washingtonpost.com/politics/2020/10/06/trailer-guide-how-pence-harris-debate/'}, {'url': 'https://www.washingtonpost.com/politics/south-carolina-mail-ballots-signature/2020/10/06/65bd6630-07e8-11eb-859b-f9c27abe638d_story.htm

In [19]:
crawler.all_json_to_send

[{'url': 'https://www.washingtonpost.com/politics/white-house-signals-stronger-coronavirus-precautions-but-trump-continues-to-resist/2020/10/06/05cd0be4-07e9-11eb-859b-f9c27abe638d_story.html'},
 {'url': 'https://www.washingtonpost.com/politics/senate-north-carolina-cunningham-democrats/2020/10/06/103de8fe-07ea-11eb-a166-dc429b380d10_story.html'},
 {'url': 'https://www.washingtonpost.com/politics/2020/10/02/visual-timeline-trumps-movements-before-his-positive-coronavirus-test/'},
 {'url': 'https://www.washingtonpost.com/politics/2020/10/02/latest-coronavirus-test-results-trumps-advisers-allies/'},
 {'url': 'https://www.washingtonpost.com/politics/courts_law/supreme-court-fbi-no-fly-list/2020/10/06/1d814b7c-07ff-11eb-859b-f9c27abe638d_story.html'},
 {'url': 'https://www.washingtonpost.com/politics/2020/10/06/trailer-guide-how-pence-harris-debate/'},
 {'url': 'https://www.washingtonpost.com/politics/south-carolina-mail-ballots-signature/2020/10/06/65bd6630-07e8-11eb-859b-f9c27abe638d_sto

### 4. Testing

Para verificar que el crawler funciona, implementamos dos tests:

In [20]:
#el crawler debe devolver una lista no vacia
assert(len(crawler.all_json_to_send)>0) 

In [21]:
#lo que devuelve el crawler deben ser URLs correctas
assert validators.url(crawler.all_json_to_send[0]["url"])

## Parte 2: Scraping

Recopilación del contenido y de metadatos (título, fecha) de las noticias de prensa

### 1. URL _seed_ 

In [22]:
SEED_SCRAP = 'https://www.washingtonpost.com/politics/courts_law/supreme-court-kim-davis-same-sex-marriage/2020/10/05/cd5a74d2-0710-11eb-9be6-cf25fb429f1a_story.html'

Para iniciar el proceso de scraping, necesitamos una página inicial que llamamos 'SEED'. Es la página que contiene una noticia.


### 2. Class _Scraper_

La clase _Scraper_ se encarga de enviar una consulta HTTP (GET) a la pagina SEED. Luego analiza la respuesta HTML para obtener el contenido de la noticia y sus metadatos.

Se requiere definir la ruta HTML para obtener los datos que nos interesan.

In [23]:
QUERY_TITLE = "//h1[@data-qa='headline']"
QUERY_CONTENT = "//div[@class='article-body']//p"
QUERY_DATE = "//div[@class='display-date']"

In [24]:
import json
import w3lib.html
import random
import os
import datetime
import requests
import string
import calendar
import html
import locale 
from requests_html import HTMLSession

In [25]:
class Scraper:
    
    USER_AGENT_LIST = [
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
        "Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/19.77.34.5 Safari/537.1",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.9 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.36 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"
    ]



    query_extract_title = QUERY_TITLE
    query_extract_text =  QUERY_CONTENT
    query_extract_date = QUERY_DATE

    def _init_(self):
        pass

    def clean_text(self, html_text):
        text_without_tags = w3lib.html.remove_tags(html_text)
        text_without_escape_chars = w3lib.html.replace_escape_chars(text_without_tags)
        text_without_escape_chars = html.unescape(text_without_escape_chars)
        text_without_whitespace = text_without_escape_chars.strip()
        return text_without_whitespace

    def replace_month_name_to_number(self,date_string):
        lower_string = date_string.lower()
        for month_id in range(1, 13):
            if(lower_string.find(calendar.month_name[month_id].lower())!=-1):
                if(month_id>=10):
                    return (lower_string.replace(calendar.month_name[month_id].lower(),str(month_id)))
                else:
                    return (lower_string.replace(calendar.month_name[month_id].lower(),"0"+str(month_id)))
            if(lower_string.find(calendar.month_abbr[month_id].lower())!=-1):
                if(month_id>=10):
                    return (lower_string.replace(calendar.month_abbr[month_id].lower(),str(month_id)))
                else:
                    return (lower_string.replace(calendar.month_abbr[month_id].lower(),"0"+str(month_id)))
                    
    #se puede cambiar
    def format_date(self,date):
        only_date = date.split('at')[0]
        only_date_with_numbers = self.replace_month_name_to_number(only_date)
        temp_date = datetime.datetime.strptime(only_date_with_numbers, '%m %d, %Y ')
        year_month_day_format = datetime.date.strftime(temp_date, "%Y-%m-%d")
        date = year_month_day_format
        return(date)

    #se puede cambiar
    def parse(self, response):
        self.news_text = []
        self.message_to_send = []
        self.scraped_url = response.url
        self.news_title = response.html.xpath(self.query_extract_title)[0].text
        self.date = response.html.xpath(self.query_extract_date)[0].text

        self.news_title = self.clean_text(self.news_title)
        self.date = self.clean_text(self.date)
        self.date = self.format_date(self.date)
        
        news_text_extracted_response = response.html.xpath(self.query_extract_text)
        for part_of_body in news_text_extracted_response:
            stripped_html_tags = w3lib.html.remove_tags(part_of_body.text)
            text_cleaned = self.clean_text(stripped_html_tags)
            self.news_text.append(text_cleaned)

    def start_request(self,url):
        headers = {'user-agent':random.choice(self.USER_AGENT_LIST) }
        session = HTMLSession()
        response = session.get(url,headers=headers)        
        return response    

### 3. Scraping

In [26]:
scraper = Scraper()
response =scraper.start_request(SEED_SCRAP)

In [27]:
scraper.parse(response)

In [28]:
scraper.news_title

'Supreme Court will not hear Kim Davis same-sex marriage case'

In [29]:
scraper.date

'2020-10-05'

In [30]:
scraper.news_text

['The Supreme Court on Monday said it will not hear a case from a Kentucky clerk who refused to issue marriage licenses for same-sex couples, but two dissenters in the court’s landmark 2015 decision repeated their criticism of its “ruinous consequences for religious liberty.”',
 'The court turned aside a case from Kim Davis, the former Rowan County clerk who was sued after she said her religious convictions kept her from recognizing same-sex marriages, even after the Supreme Court found a constitutional right to those unions in Obergefell v. Hodges. She was briefly jailed over the matter, and her case had attracted national attention.',
 'Kentucky clerk ordered to jail for refusing to issue same-sex marriage license',
 'Davis was defeated for reelection, and sued by two same-sex couples for refusing to issue marriage certificates. Her claim of qualified immunity was rejected by a panel of the U.S. Court of Appeals for the 6th Circuit.',
 'Justices Clarence Thomas and Samuel A. Alito Jr

### 4. Testing

In [31]:
# Una función que verifica que las fechas tienen el formato correcto YYYY-MM-DD
def validate(date_text):
    try:
        datetime.datetime.strptime(date_text, "%Y-%m-%d")
        return True
    except:
        print("La fecha no tiene el formato correcto, año-mes-dia")
        return False

In [32]:
#Verificamos que el scraper devuelve algo y que las fechas tienen un formato correcto
assert(len(scraper.news_title)>0)
assert(len(scraper.news_text)>0)
assert(validate(scraper.date))